In [1]:
import numpy as np
from recstate2mode import UW_,UV_,C_,mu_,Sigma_,G_matrix

In [2]:
gamma = np.array([0,0],dtype = np.complex128)
phi = np.array([0,0],dtype = np.complex128)
zeta = np.array([0,0],dtype = np.complex128)
theta = 0
psi = 0
theta1 = 0.3
psi1 = 0.1
cutoff_dim = 2
cutoff= 2
cutoff_plus = 10


In [3]:
old_state = np.random.rand(cutoff_dim,cutoff_dim) + 1.0j*np.random.rand(cutoff_dim,cutoff_dim)
old_state  /= np.linalg.norm(old_state)

In [4]:
old_state

array([[0.42332889+0.09612759j, 0.34811227+0.26652462j],
       [0.47359364+0.51538196j, 0.28292379+0.2222144j ]])

In [5]:
W = UW_(phi,theta1,psi1)
V = UV_(theta,psi)
    
C = C_(gamma, W, zeta, V)
mu = mu_(gamma, W, zeta, V)
Sigma = Sigma_(W, zeta, V)

cutoff = old_state.shape[0]
dtype = old_state.dtype


sqrt = np.sqrt(np.arange(cutoff+1, dtype=dtype))

G_00pq = np.zeros((cutoff, cutoff), dtype=dtype)


G_00pq[0,0] = C
for q in range(1, cutoff):
    G_00pq[0,q] = (mu[3]*G_00pq[0,q-1] - Sigma[3,3]*sqrt[q-1]*G_00pq[0,q-2])/sqrt[q]


for p in range(1,cutoff):
    for q in range(0,cutoff):
        G_00pq[p,q] = (mu[2]*G_00pq[p-1,q] - Sigma[2,2]*sqrt[p-1]*G_00pq[p-2,q] - Sigma[2,3]*sqrt[q]*G_00pq[p-1,q-1])/sqrt[p]

In [6]:
Sigma

array([[ 0.        +0.j        ,  0.        +0.j        ,
        -0.95533649+0.j        ,  0.29404384-0.02950279j],
       [ 0.        +0.j        ,  0.        +0.j        ,
        -0.29404384-0.02950279j, -0.95533649+0.j        ],
       [-0.95533649+0.j        , -0.29404384-0.02950279j,
         0.        +0.j        ,  0.        +0.j        ],
       [ 0.29404384-0.02950279j, -0.95533649+0.j        ,
         0.        +0.j        ,  0.        +0.j        ]])

In [7]:
G_00pq

array([[1.+0.j, 0.-0.j],
       [0.-0.j, 0.+0.j]])

In [8]:
# R_00^jk = G_00pq * a^j b^k|old_state>

R = np.zeros((cutoff, cutoff, cutoff+1, cutoff+1), dtype=dtype)

for k in range(cutoff):
    for j in range(cutoff):
        if j==0 and k ==0:
            state = old_state

        elif j>0 and k==0:
            state = old_state[:cutoff-j, :]*sqrt[1:cutoff-j+1,None]

        elif k>0 and j==0:
            state = old_state[:, :cutoff-k]*sqrt[None,1:cutoff-k+1] 

        else:
            state = old_state[:cutoff-j, :]*sqrt[1:cutoff-j+1,None]
            state = state[:, :cutoff-k]*sqrt[None,1:cutoff-k+1]

        R[0,0,j,k] = np.sum(G_00pq[:cutoff-j,:cutoff-k]*state)
    
#R_0n^jk
for k in range(0,cutoff):
    for j in range(0,cutoff):
        for n in range(1,cutoff):
            R[0,n,j,k] = mu[1]/sqrt[n]*R[0,n-1,j,k] - Sigma[1,1]/sqrt[n]*sqrt[n-1]*R[0,n-2,j,k] - Sigma[1,2]/sqrt[n]*R[0,n-1,j+1,k] - Sigma[1,3]/sqrt[n]*R[0,n-1,j,k+1]
print("oldstate",old_state)
print(R[0,1,:cutoff,:cutoff])

for k in range(0,cutoff):
    for j in range(0,cutoff):
        for n in range(0,cutoff):
            for m in range(1,cutoff):
                R[m,n,j,k] = mu[0]/sqrt[m]*R[m-1,n,j,k] - Sigma[0,0]/sqrt[m]*sqrt[m-1]*R[m-2,n,j,k] - Sigma[0,1]*sqrt[n]/sqrt[m]*R[m-1,n-1,j,k] - Sigma[0,2]/sqrt[m]*R[m-1,n,j+1,k] - Sigma[0,3]/sqrt[m]*R[m-1,n,j,k+1]


oldstate [[0.42332889+0.09612759j 0.34811227+0.26652462j]
 [0.47359364+0.51538196j 0.28292379+0.2222144j ]]
[[0.52606276+0.13258931j 0.12164122+0.04075511j]
 [0.40442154+0.0918342j  0.        +0.j        ]]


In [9]:
R[:,:,:cutoff,:cutoff]

array([[[[ 0.42332889+0.09612759j,  0.42332889+0.09612759j],
         [ 0.42332889+0.09612759j,  0.42332889+0.09612759j]],

        [[ 0.52606276+0.13258931j,  0.12164122+0.04075511j],
         [ 0.40442154+0.0918342j ,  0.        +0.j        ]]],


       [[[ 0.27710825+0.07605785j,  0.40442154+0.0918342j ],
         [-0.12731328-0.01577634j,  0.        +0.j        ]],

        [[ 0.34938841+0.07933753j,  0.        +0.j        ],
         [ 0.        +0.j        ,  0.        +0.j        ]]]])

In [10]:
R[:,:,0,0]

array([[0.42332889+0.09612759j, 0.52606276+0.13258931j],
       [0.27710825+0.07605785j, 0.34938841+0.07933753j]])

In [11]:
G = G_matrix(gamma, phi, theta1, psi1, zeta, theta, psi,cutoff)

In [13]:
np.einsum("cd,abcd->ab",old_state,G)

array([[0.42332889+0.09612759j, 0.45661643+0.42013792j],
       [0.3422178 +0.42426355j, 0.23350708+0.18340146j]])

In [14]:
np.sum(G[0,0,:,:]*old_state)

(0.42332889241713423+0.0961275926492424j)

In [15]:
np.sum(G[0,1,:,:]*old_state)

(0.45661643497875326+0.4201379193090098j)

In [16]:
np.sum(G[1,0,:,:]*old_state)

(0.3422178032264355+0.42426354964630786j)